In [1]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import httpx
import openai_setup

endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
deployed_model = os.environ["DEPLOYMENT_NAME"]
print(f"Endpoint: {endpoint}. Model: {deployed_model}") 

Endpoint: https://anildwaextopenai3.openai.azure.com/. Model: gpt-35-turbo-1106-ft-38935655ea844c37a954bff664df4358-aml_2


token_provider = get_bearer_token_provider(
            DefaultAzureCredential(),
            "https://cognitiveservices.azure.com/.default"
        )

token_provider()

In [2]:
from openai import AzureOpenAI

client = AzureOpenAI(
   api_version="2023-03-15-preview",
   azure_deployment=deployed_model,
   azure_endpoint=endpoint,
   #azure_ad_token_provider=token_provider,
)

def getOpenAIResp(userQuery, systemMessage, deployed_model='gpt-4-turbo'):
    completion = client.chat.completions.create(
            model=deployed_model,
            messages=[
                {
                    "role": "system",
                    "content": systemMessage
                },
                {
                    "role": "user",
                    "content": userQuery
                }
            ],
            temperature=0,
            max_tokens=4000,
            stream=False)
    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [3]:
systemPromptTemplate = """
You are a machine learning expert agent whose primary goal is to help users with questions with Azure Machine Learning. 
You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to Azure Machine Learning.
Answer based on the context provided to you. If you cannot answer based on the context, say that you cannot answer this question.

"""
question = "tell me a joke"
questionResponse = getOpenAIResp(question, systemPromptTemplate)
print(questionResponse)


I cannot answer this question.


In [4]:
def combineDocuments(docs, document_separator="\n\n"):
    # Convert each document in docs to a string in case they are not already.
    # This step might be redundant if docs are guaranteed to be strings.
    doc_strings = [str(doc) for doc in docs]
    
    # Use join() method to concatenate doc_strings with document_separator.
    combined_document = document_separator.join(doc_strings)
    
    return combined_document


In [6]:
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings
chroma_db = Chroma( persist_directory="./chroma",embedding_function=AzureOpenAIEmbeddings(
    #azure_ad_token_provider=token_provider,
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    azure_endpoint=endpoint,
    #http_client=httpx.Client(
    # verify=False
   #)
   ))


def getSearchResults(userQuery, numSearchResults=5):
    embedding_vector = AzureOpenAIEmbeddings().embed_query(userQuery)
    docs = chroma_db.similarity_search_by_vector(embedding_vector, k=numSearchResults)
    return combineDocuments(docs[0]), docs

In [8]:
instruction = """
        You only provide factual answers to queries, and do not provide answers that are not related to Azure Machine Learning.
        Answer based on the context provided to you. If you cannot answer based on the context, say that you cannot answer this question.
        """

### remove instruction for FT

In [9]:
def getLLMResponseWithRetriever(userQuery, docs, numSearchResults=5, deployed_model='gpt-4-turbo'):
    systemPromptTemplate = """
        You are a machine learning expert agent whose primary goal is to help users with questions with Azure Machine Learning. 
        {instruction}
        """
    userMessage = f""" 
    <Context>
    {docs}
    </Context>

    <Few_Shot_Example>

    UserQuery: {userQuery}
    """
    response = getOpenAIResp(userMessage, systemPromptTemplate, deployed_model)
    return response

In [16]:
query = f"""what is compute pool in databricks?"""
combinedDocs, docs = getSearchResults(question)
Markdown(getLLMResponseWithRetriever(query, combinedDocs, deployed_model='gpt-35-turbo-1106-ft-38935655ea844c37a954bff664df4358-aml_2'))


A compute pool in Databricks is a set of virtual machines that are used to run jobs and interactive clusters. It allows you to manage the resources available to your workloads and can be configured to automatically scale based on demand.

In [15]:
query = f"""how to integrate aml with spark?"""
combinedDocs, docs = getSearchResults(question)
Markdown(getLLMResponseWithRetriever(query, combinedDocs, deployed_model='gpt-35-turbo-1106-ft-38935655ea844c37a954bff664df4358-aml_2'))

To integrate Azure Machine Learning with Apache Spark, you can use the AzureML Spark package. This package allows you to submit Spark jobs to an Azure Machine Learning compute target. You can also use the AzureML SDK to create and manage Spark environments and submit Spark jobs.

In [12]:
import json


def getResponsePerModel(deployment_name, contextRequired, fileName):
    data_path = os.path.join('data')
    with open(os.path.join(data_path, fileName), 'w',) as outfile:
        with open(os.path.join(data_path, 'aml_questions.txt')) as f:
            questions = f.readlines()
            questions = [x.strip() for x in questions]
            for index, question in enumerate(questions):
                print(question)
                outfile_format = {"question": question, "variant_id": "v1", "line_number":index + 1, "answer":"","context":[]}
                combinedDocs = ""
                docs = []
                if contextRequired:
                    combinedDocs, docs = getSearchResults(question)
                answer = getLLMResponseWithRetriever(question, combinedDocs,deployed_model=deployment_name)
                outfile_format = {"question": question, "variant_id": "v1", "line_number":index + 1, "answer": answer,"context":[str(doc.page_content) for doc in docs]}
                json.dump(outfile_format, outfile)
                outfile.write('\n')


In [17]:
#getResponsePerModel("gpt-4-turbo", True, "gpt-4-turbo-aml-results.jsonl")
#getResponsePerModel("gpt-35-turbo", True, "gpt-35-turbo-aml-results.jsonl")
getResponsePerModel("gpt-35-turbo-1106-ft-38935655ea844c37a954bff664df4358-aml_2", True, "gpt-35-turbo-ft-aml-results.jsonl")
getResponsePerModel("gpt-35-turbo-1106-ft-38935655ea844c37a954bff664df4358-aml_2", False, "gpt-35-turbo-ft-nocontext-aml-results.jsonl")

How do you initiate a new Azure ML experiment using the Azure CLI?
Describe the process of creating a new compute instance via the Azure ML CLI. What parameters are necessary?
Explain how to submit a training job using Python and the AML SDK v2. Include an example of specifying a compute target.
What is the command to list all the models registered in an Azure ML Workspace using the Azure CLI?
Provide an example of how to use the AML SDK v2 to stream logs from a running experiment in Python.
How can you configure an AutoML run for a classification task using the AML SDK v2, including specifying primary metric and training data?
Describe the steps to retrieve the best model from an AutoML run using the AML SDK v2.
What are the AutoML capabilities for handling imbalanced data in Azure ML?
Explain how to use AutoML for forecasting time series data. What specific settings should be adjusted for time series problems?
How do you create a custom deep learning environment with specific Python 

: 